# UTH-BERT

In [2]:
! pip install transformers evaluate seqeval

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 kB 4.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 3.8 MB/s eta 0:00:00a 0:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16165 sha256=5ef81513a99cb5962be72efc474778fbd5baf8e9e159bfc35d143e9df267eb77
  Stored in directory: /home/ec2-user/.cache/pip/wheels/9c/d6/00/1ccfd5a7466a94774e00022683d4b028836032dfb85007822b
Successfully built seqeval


In [3]:
import pickle
import random
import pandas as pd
import numpy as np
import torch
import ast
import os
import evaluate
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from transformers import BertForTokenClassification, Trainer, TrainingArguments, WEIGHTS_NAME, CONFIG_NAME
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

2023-01-18 08:01:28.116718: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-18 08:01:28.460107: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
df = pd.read_csv('df_row_token_classification_dataset.csv')
for col in ['input_ids','token_type_ids','attention_mask','labels']:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x))

In [4]:
df[['tokens','input_ids','token_type_ids','attention_mask','labels']] #.sort_values('input_lengths',ascending = True)

,tokens,input_ids,token_type_ids,attention_mask,labels
0,"['[CLS]', '（', 'タ', '##グリ', '##ッ', '##ソ', 'による...","[2, 23, 3703, 7252, 2794, 4345, 198, 1395, 450...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, ..."
1,"['[CLS]', '（', '白血球減少', '）', '発熱', '性', '好中球減少...","[2, 23, 6157, 20, 242, 380, 3226, 15, 897, 6, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[-100, 1, 2, 2, 2, 2, 2, 0, 0, 0, -100, -100, ..."
2,"['[CLS]', '＋', '（', '１', '）', 'だっ', 'た', '、', ...","[2, 73, 23, 7, 20, 494, 19, 5, 6166, 2310, 23,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ..."
3,"['[CLS]', '＋', '－', 'に', '戻っ', 'た', '。', '蛋白尿'...","[2, 73, 22, 14, 1794, 19, 6, 6166, 2310, 23, 6...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."
4,"['[CLS]', '０', '０', '：', '２', '６', '下', '顎', '...","[2, 8, 8, 9, 12, 29, 251, 1973, 166, 5, 5406, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
...,...,...,...,...,...
7809,"['[CLS]', '食事', 'と', 'の', '関連', 'で', 'あろ', 'と'...","[2, 138, 27, 10, 1548, 21, 4333, 27, 885, 18, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7810,"['[CLS]', '飲み', '始め', 'て', 'から', '半年', 'ほど', '...","[2, 817, 1415, 13, 55, 3265, 549, 5, 6600, 198...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
7811,"['[CLS]', '骨髄', '検査', 'の', '結果', '、', '治療', '関...","[2, 3808, 182, 10, 379, 5, 185, 1548, 1, 9, 99...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, -100, 0, 0, 0, 0, ..."
7812,"['[CLS]', '高度', '##骨髄抑制', '、', '発熱性好中球', '減少症'...","[2, 3904, 12288, 5, 13330, 10145, 10, 56, 68, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## training

In [5]:
num_of_seed = 0

random.seed(num_of_seed)
np.random.seed(num_of_seed)
torch.manual_seed(num_of_seed) #cpu
torch.cuda.manual_seed_all(num_of_seed)#gpu
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [6]:
train_df,test_df = train_test_split(df, test_size=0.2, random_state=1)

In [13]:
train_df.to_csv('train_df.csv', index=False)
test_df.to_csv('test_df.csv', index=False)

In [7]:
def get_encoding(df):
    encoding={}
    encoding['input_ids']=df['input_ids'].tolist()
    encoding['token_type_ids']=df['token_type_ids'].tolist()
    encoding['attention_mask']=df['attention_mask'].tolist()    
    return encoding

In [8]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [9]:
train_encodings=get_encoding(train_df)
valid_encodings=get_encoding(test_df)
train_labels, valid_labels = train_df['labels'].tolist(),test_df['labels'].tolist()

In [10]:
train_dataset = Dataset(train_encodings, train_labels)
val_dataset = Dataset(valid_encodings, valid_labels)

In [6]:
label_list=["O", "B-AE", "I-AE"]
id2label = {
    0: "O",
    1: "B-AE",
    2: "I-AE",
}
label2id = {
    "O": 0,
    "B-AE": 1,
    "I-AE": 2,
}

In [13]:
seqeval = evaluate.load("seqeval")

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [9]:
BERT_MODEL = "UTH_BERT_BASE_512_MC_BPE_WWM_V25000_352K" 
model = BertForTokenClassification.from_pretrained(BERT_MODEL, num_labels=3, id2label=id2label, label2id=label2id)

In [19]:
training_args = TrainingArguments(
    output_dir="./result_uth",
    evaluation_strategy="epoch", #epoch  step
    save_strategy="no",#defaults to "steps"
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    #load_best_model_at_end=True
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [21]:
trainer.train()

***** Running training *****
  Num examples = 6251
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3910
  Number of trainable parameters = 104653059


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.163046,0.725507,0.749564,0.737339,0.947470
2,0.175400,0.148699,0.767523,0.769197,0.768359,0.953830
3,0.113600,0.149372,0.754195,0.784468,0.769033,0.953940
4,0.075500,0.148655,0.785092,0.785777,0.785434,0.955674
5,0.075500,0.164601,0.787549,0.789267,0.788407,0.957739
6,0.053400,0.182942,0.776728,0.789267,0.782947,0.957188
7,0.037000,0.184320,0.779123,0.814136,0.796245,0.957822
8,0.027200,0.193001,0.781250,0.807155,0.793991,0.957381
9,0.021200,0.197287,0.778381,0.813700,0.795648,0.957436
10,0.021200,0.201607,0.788823,0.806719,0.797670,0.958069


***** Running Evaluation *****
  Num examples = 1563
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1563
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1563
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1563
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1563
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1563
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1563
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1563
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1563
  Batch size = 64
***** Running Evaluation *****
  Num examples = 1563
  Batch size = 64


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=3910, training_loss=0.06619130095557484, metrics={'train_runtime': 10719.3405, 'train_samples_per_second': 5.832, 'train_steps_per_second': 0.365, 'total_flos': 4083451983797760.0, 'train_loss': 0.06619130095557484, 'epoch': 10.0})

In [41]:
# save model
output_dir="result_uth"
model.save_pretrained(output_dir)

Configuration saved in result_uth/config.json
Model weights saved in result_uth/pytorch_model.bin


In [47]:
"""
model_save_path = os.path.join(output_dir, 'model.pt')
torch.save(model.state_dict(), model_save_path)
"""

## inference

In [ ]:
# load model
output_dir="result_uth"
model = BertForTokenClassification.from_pretrained(output_dir)
#model=torch.load('result_uth/model.pt')

In [16]:
model.eval()
print()

In [64]:
display(train_df[11:12])
inputs={}
inputs['input_ids']=torch.tensor([train_encodings['input_ids'][11]])

print(train_df['tokens'][11:12].item())
print(train_df['labels'][11:12].item())

outputs = model(**inputs)
m = torch.nn.Softmax(dim=2)
o_args = m(outputs.logits)
pred=o_args.argmax(dim=2).numpy()[0]
print('pred:',pred)

,tokens,input_ids,token_type_ids,attention_mask,labels
5176,"['[CLS]', '[UNK]', '減少症', '（', 'ＰＡ', 'ＮＣ', 'Ｙ'...","[2, 1, 10145, 23, 1077, 1839, 1319, 85, 12914,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, -100, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


['[CLS]', '[UNK]', '減少症', '（', 'ＰＡ', 'ＮＣ', 'Ｙ', 'Ｔ', 'ＯＰＥＮ', 'ＩＡ', '）', 'も', 'あり', '緊急入院', '。', '播種', '##性', '##血管内', '##凝固', '症候群', '（', 'ＤＩＣ', '）', '発現', '。', '急性', '##肝', '##炎', '、', '肝機能障害', '、', '悪心', '、', '嘔吐', 'も', 'あり', '、', 'ＱＯＬ', 'スコア', 'も', '最低', 'の', '評価', '、', 'メロペネム', '開始', '。', 'リコモジュリン', '開始', '（', '５日間', '）', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [57]:
display(test_df[:1])
inputs={}
inputs['input_ids']=torch.tensor([valid_encodings['input_ids'][0]])

print(test_df['tokens'][:1].item())
print(test_df['labels'][:1].item())

outputs = model(**inputs)
m = torch.nn.Softmax(dim=2)
o_args = m(outputs.logits)
pred=o_args.argmax(dim=2).numpy()[0]
print('pred:',pred)

,tokens,input_ids,token_type_ids,attention_mask,labels
4939,"['[CLS]', '既往', '歴', '、', 'アレルギー歴', '、', '医', ...","[2, 413, 294, 5, 3872, 5, 76, 91, 644, 294, 5,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


['[CLS]', '既往', '歴', '、', 'アレルギー歴', '、', '医', '薬品', '副作用', '歴', '、', '家族', '歴', '、', '薬剤', '以外', 'の', '併用', '療法', '、', 'ＥＧＦＲ', '－', 'ＴＫ', '##Ｉ', '治療', '歴', '、', 'Ｔ', '７', '９', '０', 'Ｍ', '変異', '、', '前', '治療', '歴', '（', '放射線', '照射', '、', '化学', '療法', '、', 'その他', 'の', '治療', '）', '、', '心不全', '、', '肺水腫', 'の', '既往', '、', '心臓', '##弁', '##膜', '##疾患', '、', '先天性', '##心', '##疾患', '、', '心', '##膜', '疾患', '、', '心筋症', '、', '直近', 'の', '血圧', 'の', '変化', '／', '血圧', '治療', '##薬', 'の', '変更', '、', '補液', 'の', '変更', '、', '副作用', '診断', '時に', '癌', 'の', '進行', 'の', '兆候', '：', 'なし', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0

In [62]:
display(test_df[101:102])
inputs={}
inputs['input_ids']=torch.tensor([valid_encodings['input_ids'][100]])

print(test_df['tokens'][101:102].item())
print(test_df['labels'][101:102].item())

outputs = model(**inputs)
m = torch.nn.Softmax(dim=2)
o_args = m(outputs.logits)
pred=o_args.argmax(dim=2).numpy()[0]
print('pred:',pred)

,tokens,input_ids,token_type_ids,attention_mask,labels
6043,"['[CLS]', '肺癌', '転移', 'による', '髄', '##膜', '##播種...","[2, 2867, 1211, 198, 7334, 4788, 9559, 5, 1669...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 1, 2, 2, 2, 2, 2, 0, 1, 0, 0, 0, 0, 0, ..."


['[CLS]', '肺癌', '転移', 'による', '髄', '##膜', '##播種', '、', '硬膜炎', '（', '重篤', 'で', 'ない', '）', '、', '髄膜炎', 'の', '転帰', 'は', '不明', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', 

In [56]:
display(test_df[21:22])
inputs={}
inputs['input_ids']=torch.tensor([valid_encodings['input_ids'][21]])

print(test_df['tokens'][21:22].item())
print(test_df['labels'][21:22].item())

outputs = model(**inputs)
m = torch.nn.Softmax(dim=2)
o_args = m(outputs.logits)
pred=o_args.argmax(dim=2).numpy()[0]
print('pred:',pred)

,tokens,input_ids,token_type_ids,attention_mask,labels
4624,"['[CLS]', '感染症', 'の', '発症', 'あり', '、', 'イ', '#...","[2, 719, 10, 1051, 31, 5, 3843, 2331, 4442, 23...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 1, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


['[CLS]', '感染症', 'の', '発症', 'あり', '、', 'イ', '##ミ', '##フィ', '##ン', '##ジ', '投与', '中止', '。', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '

In [80]:
import copy
df_pred=test_df.copy()

In [81]:
for col in ['tokens']:
    df_pred[col] = df_pred[col].apply(lambda x: ast.literal_eval(x))

In [83]:
def get_pred(x):
    inputs={}
    inputs['input_ids']=torch.tensor([x])
    outputs = model(**inputs)
    m = torch.nn.Softmax(dim=2)
    o_args = m(outputs.logits)
    pred=o_args.argmax(dim=2).numpy()[0]
    return pred

df_pred['preds']=df_pred['input_ids'].apply(lambda x:get_pred(x))

In [85]:
def get_B_AE(tokens,preds):
    B_AE=[]
    if 1 in preds:
        B_AE=[tokens[i] for i in range(len(preds)) if preds[i]==1]
        return B_AE
    
df_pred['pred_B-AE']=df_pred.apply(lambda x:get_B_AE(x['tokens'],x['preds']), axis=1)            

In [88]:
def get_I_AE(tokens,preds):
    I_AE=[]
    if 2 in preds:
        I_AE=[tokens[i] for i in range(len(preds)) if preds[i]==2]
        return I_AE
    
df_pred['pred_I-AE']=df_pred.apply(lambda x:get_I_AE(x['tokens'],x['preds']), axis=1)            

In [102]:
def merge_B_and_I(B,I):
    B_and_I=[]   
    if B==None and I==None:
        return B_and_I
    if B==None and len(I)!=0:
        return I
    if len(B)!=0 and I==None:
        return B
    else:
        return B+I
    
df_pred['pred_B+I']=df_pred.apply(lambda x:merge_B_and_I(x['B-AE'],x['I-AE']), axis=1)    

In [131]:
def visualize(token,label,pred):
    df_ = pd.DataFrame({'token':token, 'label':label,'pred':pred})
    df_=df_.style.applymap(lambda x: 'background-color: green' if x > 0 else '', subset=['label','pred'])
    return df_

df_pred['visualization']=df_pred.apply(lambda x:visualize(x['tokens'],x['labels'],x['preds']), axis=1)   

In [138]:
df_pred

,tokens,input_ids,token_type_ids,attention_mask,labels,preds,pred_B-AE,pred_I-AE,pred_B+I,visualization
4939,"[[CLS], 既往, 歴, 、, アレルギー歴, 、, 医, 薬品, 副作用, 歴, 、,...","[2, 413, 294, 5, 3872, 5, 76, 91, 644, 294, 5,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",[心不全],None,[心不全],<pandas.io.formats.style.Styler object at 0x7f...
5112,"[[CLS], 死, ##因, ：, 肝不全, 。, [SEP], [PAD], [PAD]...","[2, 6814, 9970, 9, 7432, 6, 3, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-100, 0, 0, 0, 1, 0, -100, -100, -100, -100, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None,None,[],<pandas.io.formats.style.Styler object at 0x7f...
6761,"[[CLS], 血小板減少, に対して, は, 経過, 観察, で, 、, 貧血, に対して...","[2, 2977, 297, 15, 67, 98, 21, 5, 546, 297, 15...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[血小板減少, 貧血]",None,"[血小板減少, 貧血]",<pandas.io.formats.style.Styler object at 0x7f...
39,"[[CLS], １, コース, 目, は, シスプラチン, 投与, の, 際, の, ハイ,...","[2, 7, 979, 342, 15, 7469, 114, 10, 356, 10, 1...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None,None,[],<pandas.io.formats.style.Styler object at 0x7f...
299,"[[CLS], ２日, 目, に, 朝, から, 膨疹, 、, 下肢, の, かゆみ, が,...","[2, 1305, 342, 14, 205, 55, 10094, 5, 484, 10,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None,None,[],<pandas.io.formats.style.Styler object at 0x7f...
...,...,...,...,...,...,...,...,...,...,...
2963,"[[CLS], リ, ##ム, パー, ##ザ, 錠, を, 服用, 中, に, 全身倦怠感...","[2, 3179, 2975, 10192, 6128, 53, 25, 792, 87, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",[全身倦怠感],None,[全身倦怠感],<pandas.io.formats.style.Styler object at 0x7f...
4892,"[[CLS], 放射線, ##肺, ##臓, ##炎, の, 転帰, は, 回復, 。, [...","[2, 1498, 1628, 16196, 5778, 10, 3770, 15, 897...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[-100, 1, 2, 2, 2, 0, 0, 0, 0, 0, -100, -100, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None,None,[],<pandas.io.formats.style.Styler object at 0x7f...
5712,"[[CLS], 皮疹, （, 重篤, で, ない, ）, は, 回復, 。, [SEP], ...","[2, 820, 23, 6931, 21, 42, 20, 15, 897, 6, 3, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, ...","[-100, 1, 0, 0, 0, 0, 0, 0, 0, 0, -100, -100, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None,None,[],<pandas.io.formats.style.Styler object at 0x7f...
3829,"[[CLS], 右前, 頭部, 、, 側, 頭部, 、, 頭頂, 部, に, 髄, ##膜,...","[2, 6634, 587, 5, 571, 587, 5, 8253, 163, 14, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 2, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",None,None,[],<pandas.io.formats.style.Styler object at 0x7f...


In [151]:
df_pred.iloc[111,:]['visualization']

,token,label,pred
0,[CLS],-100,0
1,ＢＲ,0,0
2,##ＣＡ,0,0
3,遺伝子変異,0,0
4,陽性,0,0
5,の,0,0
6,卵巣癌,0,0
7,における,0,0
8,初回,0,0
9,化学,0,0


In [152]:
df_pred.to_csv('df_pred.csv', index=False)